##  Cofactor Swap

In [1]:
#In this script we analyse if swapping the cofactors has a postive effect on the resveratrol production
import pandas as pd
import matplotlib.pyplot as plt
from cobra.io import read_sbml_model
from cameo.strain_design.heuristic.evolutionary.objective_functions import product_yield
from cameo.strain_design.heuristic.evolutionary_based import CofactorSwapOptimization
plt.style.use('ggplot')

In [2]:
# Verify working directory and change it if needed
import os
os.getcwd()
os.chdir("C:/Users/Deborah/anaconda3/envs/cellfactorydesign/27410-group-assigment-group-4-resveratrol-in-s-cerevisiae/") #change accordingly

In [3]:
model = read_sbml_model("data/models/yeast_resv.xml")

In [4]:
#opt=model.optimize()
model.objective=model.reactions.VVVST1

In [12]:
py = product_yield(model.reactions.VVVST1, model.reactions.EX_glc__D_e)
optswap = CofactorSwapOptimization(model=model, objective_function=py)

In [14]:
%time
result=optswap.run(max_size=5)

Wall time: 0 ns
Starting optimization at Sun, 20 Nov 2022 22:11:24


HBox()

Finished after 00:00:28


In [15]:
result

,index,targets,fitness
